In [1]:
library(polyester)
library(Biostrings)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: S4Vectors
Loading required package: stats4

Attaching package: ‘S4Vectors’

The followin

In [14]:
setwd("/Users/tessa/dib-lab/kmerslay-analysis/simulate_reads")

In [15]:
data_dir="/Users/tessa/dib-lab/kmerslay-analysis/QfO_release_2019_04/Eukaryota"
genome_fasta=file.path(data_dir, "UP000005640_9606_DNA.fasta")
addl_fasta=file.path(data_dir, "UP000005640_9606_additional.fasta")
gene_fasta=file.path(data_dir, "UP000005640_9606.fasta")

In [16]:
fasta = readDNAStringSet(genome_fasta)

In [7]:
small_fasta = fasta[1:20]
writeXStringSet(small_fasta, 'test_small.fa')

In [ ]:
# let's sim at 20x coverage, uniform:
# ~20x coverage ----> reads per transcript = transcriptlength/readlength * 20
readspertx = round(20 * width(small_fasta) / 100)

In [ ]:
# build some fold changes
fold_changes = matrix(c(4,4,rep(1,18),1,1,4,4,rep(1,16)), nrow=20)
head(fold_changes)

In [ ]:
# test 20 transcripts
simulate_experiment('test_small.fa', reads_per_transcript=readspertx,
                    num_reps=c(10,10), fold_changes=fold_changes, outdir=simulate_experiment_countmat('chr22_small.fa', readmat=countmat, 
    outdir='timecourse_reads') )

### Sim with full QfO dataset

In [30]:
# lets sim using the full dataset
readspertx_QfO = round(20 * width(fasta) / 100)

In [31]:
fasta

  A DNAStringSet instance of length 20917
         width seq                                          names               
    [1]    462 ATGGGGGCGCCGCTGCTCTCT...GATCCTGTGCTTTACTCTGA tr|A0A024R161|ENS...
    [2]    195 ATGTCCAGCCACGAAGGTGGC...AGAAATCTGGCAAAAAATAA tr|A0A024R1R8|ENS...
    [3]    546 ATGATGAAGTTCAAGCCCAAC...TGCCATCTAGTGTAAGATAG sp|A0A024RBG1|ENS...
    [4]    390 ATGGAGACAGTGGTCACAACT...TCTACATCTGCAGTGCTAGA tr|A0A075B6H5|ENS...
    [5]    348 ATGGAAGCCCCAGCTCAGCTT...AGCAGGATTATAACTTACCT tr|A0A075B6H7|ENS...
    ...    ... ...
[20913]    291 ATGCAGGGGCAAAAGGTGCTC...GCTCCAGAGAGTGTGAATGA tr|V9GZ69|ENSP000...
[20914]    975 ATGGCCCTGCTGGCTCTGGCC...GGTACTGCAGTGGCAACTAA sp|W5XKT8|ENSP000...
[20915]    342 ATGGAGAGTAAATATAAGGAG...TTTCTGGGAATGCATGTTGA sp|W6CW81|AHI87498
[20916]    801 ATGGTGTGTCTGAAGCTCCCT...CAACAGGACTCTTGAGCTGA tr|X5D2U9|ENSP000...
[20917]    384 ATGGGCAGGAAAGAGCATGAA...TGGTCTTGTGGCGCTCCTAG tr|X6R8D5|ENSP000...

In [45]:
#count matrix seems easier than fold changes for this.

# full QfO count matrix
num_timepoints = 2
countmat = matrix(readspertx_QfO, nrow=length(fasta), ncol=num_timepoints)
#Y = scale(r, X)
# all the same rn. Could add variability like so:

# add spikes in expression at certain timepoints to certain transcripts:
#up_early = c(1,2) # transcripts #1 and #2
#up_late = c(3,4) # transcripts # 3 and #4
#countmat[up_early, 1] = 3*countmat[up_early, 1]
#countmat[up_early, 2] = round(1.5*countmat[up_early, 2])
#countmat[up_late, 4] = 6*countmat[up_late, 4]
#countmat[up_late, 5] = round(1.2*countmat[up_late, 5])


In [46]:
head(countmat)

92,92
39,39
109,109
78,78
70,70
71,71


In [51]:
simulate_experiment_countmat(genome_fasta, readmat=countmat, outdir="QfO_full") 

In [ ]:
# let's do some full human genome data

[1] "/Users/tessa/dib-lab/kmerslay-analysis/simulate_reads"

Changing other parameters and adding bias
The following parameters can be provided to simulate_experiment and simulate_experiment_countmat:

- **readlen**: Read length (default 100)
- paired: Whether the reads should be paired-end (default TRUE)
- distr: Distribution from which to draw the fragment lengths. Default is 'normal', mean=250 and sd=25. Other options are 'empirical' and 'custom'. 'empirical' means fragment lengths are drawn from a length distribution we estimated from real data, and 'custom' requires you to provide a logspline density object from which you'd like to draw the fragment lengths.
- **fraglen**: The mean fragment length, if using a normal distribution (default 250)
- fragsd: Standard devation of fragment lengths, if using a normal distribution (default 25)
- **error_model**: How should sequencing errors be simulated? The default is that sequencing errors are uniformly distributed across samples, reads, and nucleotides, at an error rate of 0.5%. Other options are 'illumina4', 'illumina5', and 'custom', where the Illumina error models were estimated from a real data set and ship with GemSIM (McElroy, Luciani, and Thomas 2012), and 'custom' allows you to use GemSIM to estimate an error model from your data set. See ?add_platform_error and/or the GemSIM paper for details. Code we used to modify GemSIM's Illumina error models for Polyester are available at our GitHub repository.
- **error_rate**: In the uniform error model, probability that the sequencer records the wrong nucleotide at any given base (default 0.005).
- **bias**: Positional bias model to use when fragmenting transcripts. By default, all fragments from a transcript are equally likely ('none'). Other choices are 'rnaf' and 'cdnaf', which mimic positional bias arising from different fragmentation protocols. See ?generate_fragments and our manuscript (Frazee et al, 2014) for details.
- **gc_bias**: sample-specific GC bias models to be used to change expression values after read numbers are assigned. We modeled transcript expression as a function of GC content for 7 biological replicates in a real data set, and shift expression values accordingly. See ?add_gc_bias for details. Ignored in simulate_experiment_countmat.
- frag_GC_bias: A sample-specific GC content bias on the fragment level instead of the transcript level. See ?simulate_experiment
- strand_specific: Whether the experiment should be strand-specific or not, default is FALSE so an unstranded experiment.
- meanmodel: Set to TRUE to estimate reads_per_transcript as a data-driven function of transcript length. Ignored in simulate_experiment_countmat.
- lib_sizes: multiplicative library size factor for each replicate in the experiment. Ignored in simulate_experiment_countmat.

# Using real data to guide simulation
We also provide a function simulate_experiment_empirical that takes a real transcript expression matrix (in FPKM or RPKM units) and corresponding annotation to simulate an experiment with abundances and differential expression fold changes similar to those given in the expression matrix. This function is compatible with the Ballgown package, or you can simply provide a transcript-by-replicate expression matrix generated with your favorite abundance estimation software.